In [338]:
# this will update the json, so use only once
%run '../manifold-users/manifold-users.ipynb'
# use absolute path so we can use this in other notebooks
all_users_filepath = "/home/db/Documents/JRPdata/manifold-users/all_users.json"


 -- API - data for 1 new users...
Updated (all) users. New users: 1. Total users found: 42338
Done updating.
Number of users created between 2023-06-25T00:00:00-08:00 and 2023-07-25T00:00:00-08:00: 1459


In [339]:
# Define types (class-like structures) and constants
class Answer:
    pass

class DpmAnswer:
    pass

class Bet:
    pass

class MultiSerializedPoint:
    pass

class SerializedPoint:
    pass

class Fees:
    pass

class JSONContent:
    pass

class GroupLink:
    pass

class ContractMetric:
    pass

class ContractMetricsByOutcome:
    pass

class ContractComment:
    pass

ENV_CONFIG = {
    'domain': 'your_domain_here'
}

def formatMoney(amount):
    return f"${amount:.2f}"

def formatPercent(percent):
    return f"{percent:.2%}"

def getLiquidity(pool):
    return sum(pool.values())

#def sum(iterable):
#    return sum(iterable)

def getDisplayProbability(contract):
    return contract.prob

class PollOption:
    pass

class AnyOutcomeType:
    pass

class AnyContractType:
    pass

class Contract:
    pass

class DPMContract:
    pass

class CPMMContract:
    pass

class CPMMMultiContract:
    pass

class BinaryContract:
    pass

class DPMBinaryContract:
    pass

class CPMMBinaryContract:
    pass

class PseudoNumericContract:
    pass

class NumericContract:
    pass

class FreeResponseContract:
    pass

class MultipleChoiceContract:
    pass

class CertContract:
    pass

class Uniswap2CertContract:
    pass

class DpmMultipleChoiceContract:
    pass

class QuadraticFundingContract:
    pass

class StonkContract:
    pass

class CPMMStonkContract:
    pass

class BountiedQuestionContract:
    pass

class PollContract:
    pass

class BinaryOrPseudoNumericContract:
    pass

class DPM:
    pass

class CPMM2:
    pass

class CPMM:
    pass

class Uniswap2:
    pass

class NonBet:
    pass

class CPMMMulti:
    pass

class Cert:
    pass

class QuadraticFunding:
    pass

class Binary:
    pass

class PseudoNumeric:
    pass

class FreeResponse:
    pass

class MultipleChoice:
    pass

class Numeric:
    pass

class Stonk:
    pass

class BountiedQuestion:
    pass

class Poll:
    pass

class MultiContract:
    pass

class OutcomeType:
    pass

class ContractParams:
    pass

class MaybeAuthedContractParams:
    pass

# Define constants
NON_BETTING_OUTCOMES = ['BOUNTIED_QUESTION']
RESOLUTIONS = ['YES', 'NO', 'MKT', 'CANCEL']
OUTCOME_TYPES = [
    'BINARY', 'MULTIPLE_CHOICE', 'FREE_RESPONSE', 'PSEUDO_NUMERIC',
    'NUMERIC', 'CERT', 'QUADRATIC_FUNDING', 'STONK', 'BOUNTIED_QUESTION', 'POLL'
]

MAX_QUESTION_LENGTH = 120
MAX_DESCRIPTION_LENGTH = 16000
MAX_TAG_LENGTH = 60
CPMM_MIN_POOL_QTY = 0.01
VISIBILITIES = ['public', 'unlisted', 'private']

# Define functions
def contractPathWithoutContract(creatorUsername, slug):
    return f"/{creatorUsername}/{slug}"

def contractUrl(contract):
    return f"https://{ENV_CONFIG['domain']}{contractPath(contract)}"

def contractPool(contract):
    if contract.mechanism == 'cpmm-1':
        return formatMoney(contract.totalLiquidity)
    elif contract.mechanism == 'cpmm-2':
        return formatMoney(getLiquidity(contract.pool))
    elif contract.mechanism == 'dpm-2':
        return formatMoney(sum(contract.pool.values()))
    elif contract.mechanism == 'cpmm-multi-1':
        total_liquidity = sum(
            getLiquidity({'YES': a.poolYes, 'NO': a.poolNo})
            for a in contract.answers
        )
        return formatMoney(total_liquidity)
    else:
        return 'Empty pool'

def getBinaryProbPercent(contract):
    return formatPercent(getDisplayProbability(contract))

def tradingAllowed(contract):
    return (
        not contract.isResolved
        and (not contract.closeTime or contract.closeTime > time.time())
        and contract.mechanism != 'none'
    )


In [340]:
import math
import time
from typing import List, Dict, Union
from functools import partial

# Define data types for the function arguments
CpmmState = Dict[str, Union[float, int]]
LimitBet = Dict[str, Union[str, float, int, None]]
Fill = Dict[str, Union[str, float, int, CpmmState, None]]
Order = Dict[str, Union[Fill, LimitBet]]

def calculateCpmmAmountToProb(state, prob, outcome):
    if prob <= 0 or prob >= 1 or math.isnan(prob):
        return float('inf')
    
    if outcome == 'NO':
        prob = 1 - prob

    y, n = state['pool']['YES'], state['pool']['NO']
    k = y ** state['p'] * n ** (1 - state['p'])

    if outcome == 'YES':
        return (((state['p'] * (prob - 1)) / ((state['p'] - 1) * prob)) ** (-state['p']) *
                (k - n * (((state['p'] * (prob - 1)) / ((state['p'] - 1) * prob)) ** state['p'])))
    else:
        return ((((1 - state['p']) * (prob - 1)) / (-state['p'] * prob)) ** (state['p'] - 1) *
                (k - y * ((((1 - state['p']) * (prob - 1)) / (-state['p'] * prob)) ** (1 - state['p']))))

"""
def sortBy(bets: List[LimitBet], key_func: callable, secondary_key_func: callable) -> List[LimitBet]:
    return sorted(bets, key=key_func, reverse=True), sorted(bets, key=secondary_key_func)
"""
# lodash equivalent
def sortBy(items, *key_funcs, reverse=False):
    # Apply each key function in reverse order (if any) to achieve the desired sorting order
    for key_func in reversed(key_funcs):
        items.sort(key=key_func, reverse=reverse)
    return items
# lodash equivalent
def sumBy(items, key):
    return sum(item[key] for item in items)
"""
def sumBy(items: List[Dict[str, float]], key: str) -> float:
    return sum(item[key] for item in items)
"""

# Define the main functions
def computeFill(amount: float, outcome: str, limitProb: float, cpmmState: CpmmState, 
                matchedBet: LimitBet, matchedBetUserBalance: float) -> Union[None, Order]:
    prob = getCpmmProbability(cpmmState['pool'], cpmmState['p'])
    
    # Only for limit orders
    limitProb = None
    if (
        limitProb is not None and
        (
            (outcome == 'YES'
                and prob >= limitProb
                and (matchedBet.get('limitProb', 1) > limitProb)
            ) or
            (outcome == 'NO'
                and prob >= limitProb
                and (matchedBet.get('limitProb', 0) < limitProb)
            )
        )
    ):
        # No fill.
        return None

    timestamp = int(time.time() * 1000)

    if not matchedBet or (
        (outcome == 'YES' and not prob >= matchedBet['limitProb']) or
        (outcome == 'NO' and not prob <= matchedBet['limitProb'])
    ):
        # Fill from pool.
        if matchedBet is None:
            limit = limitProb
        else:
            if outcome == 'YES':
                if not limitProb:
                    lp = 1
                else:
                    lp = limitProb
                limit = min(matchedBet['limitProb'], lp)
            else:
                if not limitProb:
                    lp = 0
                else:
                    lp = limitProb
                limit = max(matchedBet['limitProb'], lp)

        buyAmount = amount if limit is None else min(amount, calculateCpmmAmountToProb(cpmmState, limit, outcome))

        shares, newPool, newP, fees = calculateCpmmPurchase(cpmmState, buyAmount, outcome)
        newState = {'pool': newPool, 'p': newP}

        return {
            'maker': {
                'matchedBetId': None,
                'shares': shares,
                'amount': buyAmount,
                'state': newState,
                'fees': fees,
                'timestamp': timestamp,
            },
            'taker': {
                'matchedBetId': None,
                'shares': shares,
                'amount': buyAmount,
                'timestamp': timestamp,
            },
        }

    # Fill from matchedBet.
    amountRemaining = matchedBet['orderAmount'] - matchedBet['amount']
    matchableUserBalance = max(matchedBetUserBalance, 0) if matchedBetUserBalance is not None else amountRemaining
    amountToFill = min(amountRemaining, matchableUserBalance)

    shares = min(
        amount / (matchedBet['limitProb'] if outcome == 'YES' else 1 - matchedBet['limitProb']),
        amountToFill / (1 - matchedBet['limitProb'] if outcome == 'YES' else matchedBet['limitProb'])
    )

    maker = {
        'bet': matchedBet,
        'matchedBetId': 'taker',
        'amount': shares * (1 - matchedBet['limitProb'] if outcome == 'YES' else matchedBet['limitProb']),
        'shares': shares,
        'timestamp': timestamp,
    }
    taker = {
        'matchedBetId': matchedBet['id'],
        'amount': shares * (matchedBet['limitProb'] if outcome == 'YES' else 1 - matchedBet['limitProb']),
        'shares': shares,
        'timestamp': timestamp,
    }
    return {'maker': maker, 'taker': taker}

def bet_wrapper_limitProb(bet, outcome):
    return bet['limitProb'] if outcome == 'YES' else -bet['limitProb']

def bet_wrapper_createdTime(bet):
    return bet['createdTime']

def computeFills(state: CpmmState, outcome: str, betAmount: float, limitProb: float, 
                 unfilledBets: List[LimitBet], balanceByUserId: Dict[str, float]) -> Dict:
    # Define the 'noFees' object which was not provided in the original code
    noFees = 0.0  # Replace with the actual value of 'noFees' if known

    if not isinstance(betAmount, (int, float)) or isinstance(limitProb, float) and isNaN(limitProb):
        raise ValueError(f"Invalid bet amount: {betAmount}")
    if limitProb is not None and isNaN(limitProb):
        raise ValueError(f"Invalid limitProb: {limitProb}")

    now = int(time.time() * 1000)
    """    sortedBets, _ = sortBy(
        [bet for bet in unfilledBets if bet['outcome'] != outcome and (('expiresAt' not in bet) or bet['expiresAt'] > now)],
        key_func=lambda bet: bet['limitProb'] if outcome == 'YES' else -bet['limitProb'],
        secondary_key_func=lambda bet: bet['createdTime']
    )
    """
    
    items = [bet for bet in unfilledBets if bet['outcome'] != outcome and (('expiresAt' not in bet) or bet['expiresAt'] > now)]
    sortedBets = sortBy(
        items,
        partial(bet_wrapper_limitProb, outcome=outcome),
        partial(bet_wrapper_createdTime)
    )

    
    takers = []
    makers = []
    ordersToCancel = []

    amount = betAmount
    cpmmState = {'pool': state['pool'], 'p': state['p']}
    totalFees = noFees
    currentBalanceByUserId = {k: v for k, v in balanceByUserId.items()}  # Create a copy of balanceByUserId

    i = 0
    while True:
        if i < len(sortedBets):
            matchedBet = sortedBets[i]
            ubalance = currentBalanceByUserId.get(matchedBet['userId'], 0)
        else:
            matchedBet = None
            ubalance = None
        
        fill = computeFill(amount, outcome, limitProb, cpmmState, matchedBet, ubalance)
        if not fill:
            break

        taker, maker = fill['taker'], fill['maker']

        if maker['matchedBetId'] is None:
            # Matched against pool.
            cpmmState = maker['state']
            totalFees += maker['fees']
            takers.append(taker)
        else:
            # Matched against bet.
            i += 1
            userId = maker['bet']['userId']
            makerBalance = currentBalanceByUserId[userId]
            if makerBalance is not None:
                if maker['amount'] > 0:
                    currentBalanceByUserId[userId] = makerBalance - maker['amount']
                adjustedMakerBalance = currentBalanceByUserId[userId]
                if adjustedMakerBalance is not None and adjustedMakerBalance <= 0:
                    # Now they've insufficient balance. Cancel maker bet.
                    ordersToCancel.append(maker['bet'])
            if maker['amount'] == 0:
                continue

            takers.append(taker)
            makers.append(maker)

        amount -= taker['amount']

        if amount == 0:
            break

    return {
        'takers': takers,
        'makers': makers,
        'totalFees': totalFees,
        'cpmmState': cpmmState,
        'ordersToCancel': ordersToCancel,
    }

def computeCpmmBet(cpmmState: CpmmState, outcome: str, betAmount: float, limitProb: float, 
                   unfilledBets: List[LimitBet], balanceByUserId: Dict[str, float]) -> Dict:
    result = computeFills(cpmmState, outcome, betAmount, limitProb, unfilledBets, balanceByUserId)
    takers, makers, afterCpmmState, ordersToCancel = result['takers'], result['makers'], result['cpmmState'], result['ordersToCancel']
    probBefore = getCpmmProbability(cpmmState['pool'], cpmmState['p'])
    probAfter = getCpmmProbability(afterCpmmState['pool'], afterCpmmState['p'])

    takerAmount = sumBy(takers, 'amount')
    takerShares = sumBy(takers, 'shares')
    isFilled = betAmount == takerAmount

    return {
        'orderAmount': betAmount,
        'amount': takerAmount,
        'shares': takerShares,
        'isFilled': isFilled,
        'fills': takers,
        'probBefore': probBefore,
        'probAfter': probAfter,
        'makers': makers,
        'ordersToCancel': ordersToCancel,
        **afterCpmmState,
    }

# Note: Some utility functions and constant variables are not provided in the original code.
# You may need to replace those with actual implementations or values depending on the context of your use case.

def getBinaryCpmmBetInfo(contract: Dict, outcome: str, betAmount: float, limitProb: float, 
                         unfilledBets: List[LimitBet], balanceByUserId: Dict[str, float], expiresAt=None) -> Dict:
    cpmmState = {'pool': contract['pool'], 'p': contract['p']}
    result = computeCpmmBet(cpmmState, outcome, betAmount, limitProb, unfilledBets, balanceByUserId)
    if 'visibility' in contract:
        visibility = contract['visibility']
    else:
        visibility = None
    newBet = {
        'orderAmount': result['orderAmount'],
        'amount': result['amount'],
        'shares': result['shares'],
        'limitProb': limitProb,
        'isFilled': result['isFilled'],
        'isCancelled': False,
        'fills': result['fills'],
        'contractId': contract['id'],
        'outcome': outcome,
        'probBefore': result['probBefore'],
        'probAfter': result['probAfter'],
        'loanAmount': 0,
        'createdTime': int(time.time() * 1000),
        'fees': 0.0,  # Replace with the actual value of 'fees' if known
        'isAnte': False,
        'isRedemption': False,
        'isChallenge': False,
        'visibility': visibility,
        'expiresAt': expiresAt,
    }

    return {
        'newBet': newBet,
        'newPool': result['pool'],
        'newP': result['p'],
        'makers': result['makers'],
        'ordersToCancel': result['ordersToCancel'],
    }


In [341]:
import json
jsonstr = r"""{"id":"irIFAGnxjQynchTNRiMx","creatorId":"APtmK3Sep3TJB92vIgWeSmAEwVa2","creatorUsername":"parhizj","creatorName":"JRP","createdTime":1693603148826,"creatorAvatarUrl":"https://firebasestorage.googleapis.com/v0/b/mantic-markets.appspot.com/o/user-images%2Fparhizj%2FH7gwcxOdlA.jpg?alt=media&token=30c22eec-0b17-4d18-8a85-f1db3ce3acf8","closeTime":1696132800000,"question":"Arctic Sea Ice Volume for September 2023 >= 4,900 km^3","url":"https://manifold.markets/parhizj/arctic-sea-ice-volume-for-september","pool":{"NO":289.4503968614656,"YES":159.27045244798285},"probability":0.7418064003624136,"p":0.6125392284511574,"totalLiquidity":210,"outcomeType":"BINARY","mechanism":"cpmm-1","volume":669.7121464988467,"volume24Hours":163.99625445444414,"isResolved":false,"lastUpdatedTime":1695338905951,"description":{"type":"doc","content":[{"type":"paragraph","content":[{"text":"Will the monthly Arctic Sea Ice Volume for September 2023, as reported from the PIOMAS reanalysis, be greater than or equal to 4,900 km^3?","type":"text"}]},{"type":"paragraph","content":[{"text":"Criteria:","type":"text"}]},{"type":"paragraph","content":[{"text":"The market will resolve YES if the number reported from PIOMAS (presently at ","type":"text"},{"text":"http://psc.apl.uw.edu/research/projects/arctic-sea-ice-volume-anomaly/data/","type":"text","marks":[{"type":"link","attrs":{"href":"http://psc.apl.uw.edu/research/projects/arctic-sea-ice-volume-anomaly/data/","class":"break-anywhere hover:underline hover:decoration-primary-400 hover:decoration-2","target":"_blank"}},{"type":"bold"}]},{"text":") for the \"Monthly Ice Volume\" is greater than or equal to 4,900 km^3. I will use the csv data linked in the above URL. Please note the data is in units of 10^3 km^3, so a value of 4.900 in the .csv file equals 4,900 km^3; e.g., a value of >= 4.900 in the csv file for September 2023 will resolve YES; a value < 4.900 will resolve NO).","type":"text"}]},{"type":"paragraph"},{"type":"paragraph","content":[{"text":"Motivation:","type":"text"}]},{"type":"paragraph","content":[{"text":"Quote from the PIOMAS [1] web page: \"Sea ice volume is an important climate indicator. It depends on both ice thickness and extent and therefore more directly tied to climate forcing than extent alone.\"","type":"text"}]},{"type":"paragraph"},{"type":"paragraph","content":[{"text":"Resolution Timeframe:","type":"text"}]},{"type":"paragraph","content":[{"text":"This market will close on October 1. I will try to resolve the market within a day of the data for September 2023 becoming available. To keep the market somewhat timely, I will give up to (roughly) three weeks for the data to be published; specifically, if the data for the month of September 2023 is not available for download by October 21, 2023 12:00 PM EST, I will resolve this market to N/A then.","type":"text"}]},{"type":"paragraph"},{"type":"paragraph","content":[{"text":"Comments:","type":"text"}]},{"type":"paragraph","content":[{"text":"Keep in mind that PIOMAS is a reanalysis, so this market's question is ultimately predicting the model's estimate for arctic ice volume (see the PIOMAS \"Purpose\" section for more information).","type":"text"}]},{"type":"paragraph","content":[{"text":"A few references for general knowledge:","type":"text"}]},{"type":"paragraph","content":[{"text":"[1] PIOMAS:","type":"text"}]},{"type":"paragraph","content":[{"text":"http://psc.apl.uw.edu/research/projects/arctic-sea-ice-volume-anomaly/","type":"text","marks":[{"type":"link","attrs":{"href":"http://psc.apl.uw.edu/research/projects/arctic-sea-ice-volume-anomaly/","class":"break-anywhere hover:underline hover:decoration-primary-400 hover:decoration-2","target":"_blank"}},{"type":"bold"}]}]},{"type":"paragraph","content":[{"text":"[2] Some further nice info graphics on sea ice, courtesy of Zachary Labe:","type":"text"}]},{"type":"paragraph","content":[{"text":"https://zacklabe.com/arctic-sea-ice-volumethickness/","type":"text","marks":[{"type":"link","attrs":{"href":"https://zacklabe.com/arctic-sea-ice-volumethickness/","class":"break-anywhere hover:underline hover:decoration-primary-400 hover:decoration-2","target":"_blank"}},{"type":"bold"}]}]},{"type":"paragraph","content":[{"text":"[3] Sea Ice Prediction Network (providing forecasts for sea ice extent, analysis, and many other things):","type":"text"}]},{"type":"paragraph","content":[{"text":"https://www.arcus.org/sipn","type":"text","marks":[{"type":"link","attrs":{"href":"https://www.arcus.org/sipn","class":"break-anywhere hover:underline hover:decoration-primary-400 hover:decoration-2","target":"_blank"}},{"type":"bold"}]}]},{"type":"linkPreview","attrs":{"id":"8a2d90d5-d0e4-4c01-bf0d-3f28264bc4e4","url":"https://zacklabe.com/arctic-sea-ice-volumethickness/","image":"https://zacklabe.files.wordpress.com/2023/08/meansit_jul23.gif","title":"Arctic: Sea-Ice Thickness/Volume","inputKey":"create marketundefined","deleteNode":null,"description":"My visualizations: Arctic Climate Seasonality and Variability Arctic Sea Ice Extent and Concentration Arctic Sea Ice Volume and Thickness Arctic Temperatures Antarctic Sea Ice Extent and Concentration Climate Change Indicators Climate model projections compared to observations in the Arctic Global Sea Ice Extent and Concentration Polar Climate Change Figures Climate Viz of the Month My…","deleteCallback":null,"hideCloseButton":false}}]},"groupSlugs":["climate"],"textDescription":"Will the monthly Arctic Sea Ice Volume for September 2023, as reported from the PIOMAS reanalysis, be greater than or equal to 4,900 km^3?\n\nCriteria:\n\nThe market will resolve YES if the number reported from PIOMAS (presently at http://psc.apl.uw.edu/research/projects/arctic-sea-ice-volume-anomaly/data/) for the \"Monthly Ice Volume\" is greater than or equal to 4,900 km^3. I will use the csv data linked in the above URL. Please note the data is in units of 10^3 km^3, so a value of 4.900 in the .csv file equals 4,900 km^3; e.g., a value of >= 4.900 in the csv file for September 2023 will resolve YES; a value < 4.900 will resolve NO).\n\nMotivation:\n\nQuote from the PIOMAS [1] web page: \"Sea ice volume is an important climate indicator. It depends on both ice thickness and extent and therefore more directly tied to climate forcing than extent alone.\"\n\nResolution Timeframe:\n\nThis market will close on October 1. I will try to resolve the market within a day of the data for September 2023 becoming available. To keep the market somewhat timely, I will give up to (roughly) three weeks for the data to be published; specifically, if the data for the month of September 2023 is not available for download by October 21, 2023 12:00 PM EST, I will resolve this market to N/A then.\n\nComments:\n\nKeep in mind that PIOMAS is a reanalysis, so this market's question is ultimately predicting the model's estimate for arctic ice volume (see the PIOMAS \"Purpose\" section for more information).\n\nA few references for general knowledge:\n\n[1] PIOMAS:\n\nhttp://psc.apl.uw.edu/research/projects/arctic-sea-ice-volume-anomaly/\n\n[2] Some further nice info graphics on sea ice, courtesy of Zachary Labe:\n\nhttps://zacklabe.com/arctic-sea-ice-volumethickness/\n\n[3] Sea Ice Prediction Network (providing forecasts for sea ice extent, analysis, and many other things):\n\nhttps://www.arcus.org/sipn\n\n[link preview]"}"""
bets_str = r"""[{"id":"UM82Sloa5PLQg6Uy2xmp","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":-59.14265782048044,"isSale":true,"shares":-76.81405682435775,"timestamp":1695338905634,"matchedBetId":null}],"isApi":false,"amount":-59.14265782048044,"isAnte":false,"shares":-76.81405682435775,"userId":"KyL8Ne9ytVZOcOcfHFxF9DacOgL2","outcome":"YES","isFilled":true,"userName":"Christopher Randles","probAfter":0.7418064003624136,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":-15.175165329156183,"probBefore":0.7955815260196704,"visibility":"public","createdTime":1695338905634,"isCancelled":false,"isChallenge":false,"orderAmount":-59.14265782048044,"isRedemption":false,"userUsername":"ChristopherRandles","userAvatarUrl":"https://lh3.googleusercontent.com/a/AGNmyxY3_PAyuLB3J_OqBJQ5T3NjkNLcX7WEzxb7X0Fw=s96-c"},{"id":"EqL8fTNtxWiuHjmbvSyN","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":50,"shares":64.6240144801119,"timestamp":1695322020774,"matchedBetId":null}],"isApi":false,"amount":50,"isAnte":false,"shares":64.6240144801119,"userId":"UVBIAmPaM9StgSeKzodZzPtjAqE3","outcome":"YES","isFilled":true,"userName":"Sonia Albrecht","probAfter":0.7955815260196705,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":0,"probBefore":0.7500000000000002,"visibility":"public","createdTime":1695322020774,"isCancelled":false,"isChallenge":false,"orderAmount":50,"isRedemption":false,"userUsername":"SoniaAlbrecht","userAvatarUrl":"https://lh3.googleusercontent.com/a-/AFdZucpYiTMRqyXfe1fqledGrEzHwZffv8xdrmckFVTDiw=s96-c"},{"id":"V7kkZAf8KScrx1tI3nbm","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":4.853596633963692,"shares":20.08173762345507,"timestamp":1695319199826,"matchedBetId":null}],"isApi":true,"amount":4.853596633963692,"isAnte":false,"shares":20.08173762345507,"userId":"BhNkw088bMNwIFF2Aq5Gg9NTPzz1","outcome":"NO","isFilled":false,"userName":"Acceleration","limitProb":0.75,"probAfter":0.7500000000000001,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":0,"probBefore":0.7664031488055704,"visibility":"public","createdTime":1695319199826,"isCancelled":true,"isChallenge":false,"orderAmount":100,"isRedemption":false,"userUsername":"acc","userAvatarUrl":"https://firebasestorage.googleapis.com/v0/b/mantic-markets.appspot.com/o/user-images%2FAccelerationBot%2FtGI0YIHNo5.44?alt=media&token=b4b3edf9-839c-4f1f-941e-82b308eedd5e"},{"id":"zs1dXEeWWwYd8MnAZ7w5","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":40,"shares":53.78170620371782,"timestamp":1695319199601,"matchedBetId":null}],"isApi":false,"amount":40,"isAnte":false,"shares":53.78170620371782,"userId":"UVBIAmPaM9StgSeKzodZzPtjAqE3","outcome":"YES","isFilled":true,"userName":"Sonia Albrecht","probAfter":0.7664031488055704,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":0,"probBefore":0.7195051192100477,"visibility":"public","createdTime":1695319199601,"isCancelled":false,"isChallenge":false,"orderAmount":40,"isRedemption":false,"userUsername":"SoniaAlbrecht","userAvatarUrl":"https://lh3.googleusercontent.com/a-/AFdZucpYiTMRqyXfe1fqledGrEzHwZffv8xdrmckFVTDiw=s96-c"},{"id":"oJFgU9fqoGjZGQSZCHci","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":10,"shares":14.032308118977284,"timestamp":1695319014675,"matchedBetId":null}],"isApi":false,"amount":10,"isAnte":false,"shares":14.032308118977284,"userId":"UVBIAmPaM9StgSeKzodZzPtjAqE3","outcome":"YES","isFilled":true,"userName":"Sonia Albrecht","probAfter":0.7195051192100477,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":0,"probBefore":0.7056629260539171,"visibility":"public","createdTime":1695319014675,"isCancelled":false,"isChallenge":false,"orderAmount":10,"isRedemption":false,"userUsername":"SoniaAlbrecht","userAvatarUrl":"https://lh3.googleusercontent.com/a-/AFdZucpYiTMRqyXfe1fqledGrEzHwZffv8xdrmckFVTDiw=s96-c"},{"id":"Z3QuZ2FAX9hmn0GlFXTO","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":10,"shares":14.33837941851209,"timestamp":1695156854781,"matchedBetId":null}],"isApi":false,"amount":10,"isAnte":false,"shares":14.33837941851209,"userId":"g5WnCeckQeQO6FnxdD25Y3mUqnG3","outcome":"YES","isFilled":true,"userName":"Jason Hunger","probAfter":0.7056629260539168,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":0.784,"probBefore":0.6890481785047832,"visibility":"public","createdTime":1695156854781,"isCancelled":false,"isChallenge":false,"orderAmount":10,"isRedemption":false,"userUsername":"JasonHunger","userAvatarUrl":"https://lh3.googleusercontent.com/a/AAcHTtcbf7bUr1UpWpVY_jp-694da81pk1ybyQNsKhmisLiO=s96-c"},{"id":"CQGymvr2u49fN6aR86WH","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":10,"shares":14.702628780448663,"timestamp":1695139643453,"matchedBetId":null}],"isApi":true,"amount":10,"isAnte":false,"shares":14.702628780448663,"userId":"k0mod5J32paw4ucMw8002tVo7wH3","outcome":"YES","isFilled":true,"userName":"d6e","probAfter":0.6890481785047832,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":0.784,"probBefore":0.6710995382298051,"visibility":"public","createdTime":1695139643453,"isCancelled":false,"isChallenge":false,"orderAmount":10,"isRedemption":false,"userUsername":"d6e","userAvatarUrl":"https://firebasestorage.googleapis.com/v0/b/mantic-markets.appspot.com/o/user-images%2Fd6e%2Flain.gif?alt=media&token=ca23f2fc-efee-4403-a777-39eccbf03181"},{"id":"EhXh9JLUthIfSsl7zVI5","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":-58.15184575074767,"isSale":true,"shares":-81,"timestamp":1695131285592,"matchedBetId":null}],"isApi":false,"amount":-58.15184575074767,"isAnte":false,"shares":-81,"userId":"KyL8Ne9ytVZOcOcfHFxF9DacOgL2","outcome":"YES","isFilled":true,"userName":"Christopher Randles","probAfter":0.6710995382298052,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":-15.852052381094854,"probBefore":0.76,"visibility":"public","createdTime":1695131285592,"isCancelled":false,"isChallenge":false,"orderAmount":-58.15184575074767,"isRedemption":false,"userUsername":"ChristopherRandles","userAvatarUrl":"https://lh3.googleusercontent.com/a/AGNmyxY3_PAyuLB3J_OqBJQ5T3NjkNLcX7WEzxb7X0Fw=s96-c"},{"id":"vb9OJaeaewvEXsDPYfkO","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"amount":-30.68724535430562,"isAnte":false,"shares":-40.37795441356002,"userId":"BhNkw088bMNwIFF2Aq5Gg9NTPzz1","outcome":"YES","answerId":"undefined","probAfter":0.7600000000000001,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":-1.2095406414245191,"probBefore":0.7600000000000001,"visibility":"public","createdTime":1695124209059,"isChallenge":false,"isRedemption":true},{"id":"1tiutAZNppiuGOB5kKHZ","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"amount":-9.6907090592544,"isAnte":false,"shares":-40.37795441356002,"userId":"BhNkw088bMNwIFF2Aq5Gg9NTPzz1","outcome":"NO","answerId":"undefined","probAfter":0.7600000000000001,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":-1.2095406414245191,"probBefore":0.7600000000000001,"visibility":"public","createdTime":1695124209059,"isChallenge":false,"isRedemption":true},{"id":"7djLud1cON6hyy8kVMPK","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":29.791248649624777,"shares":40.37795441356002,"timestamp":1695124208916,"matchedBetId":null}],"isApi":true,"amount":29.791248649624777,"isAnte":false,"shares":40.37795441356002,"userId":"BhNkw088bMNwIFF2Aq5Gg9NTPzz1","outcome":"YES","isFilled":false,"userName":"Acceleration","limitProb":0.76,"probAfter":0.7600000000000001,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":0,"probBefore":0.7142338763385023,"visibility":"public","createdTime":1695124208916,"isCancelled":true,"isChallenge":false,"orderAmount":100,"isRedemption":false,"userUsername":"acc","userAvatarUrl":"https://firebasestorage.googleapis.com/v0/b/mantic-markets.appspot.com/o/user-images%2FAccelerationBot%2FtGI0YIHNo5.44?alt=media&token=b4b3edf9-839c-4f1f-941e-82b308eedd5e"},{"id":"536KKaZrrG9f2qHFicvK","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":20,"shares":83.30181743012878,"timestamp":1695124208681,"matchedBetId":null}],"isApi":false,"amount":20,"isAnte":false,"shares":83.30181743012878,"userId":"2yBNXibHeMYGUyH0itSEUPLK1lh2","outcome":"NO","isFilled":true,"userName":"Blackle Mori","probAfter":0.7142338763385023,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":1.568,"probBefore":0.7999999999999999,"visibility":"public","createdTime":1695124208681,"isCancelled":false,"isChallenge":false,"orderAmount":20,"isRedemption":false,"userUsername":"blackle","userAvatarUrl":"https://lh3.googleusercontent.com/a/AAcHTtcT_8UllAUoQqGv7PYMmoAr4y7ISZuhuKJxiESYmHJZurQ=s96-c"},{"id":"yh64lCBvAEv3082ddbQD","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":2.3638179180241483,"shares":12.176416771354155,"timestamp":1695007885799,"matchedBetId":null}],"isApi":true,"amount":2.3638179180241483,"isAnte":false,"shares":12.176416771354155,"userId":"BhNkw088bMNwIFF2Aq5Gg9NTPzz1","outcome":"NO","isFilled":false,"userName":"Acceleration","limitProb":0.8,"probAfter":0.8,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":0,"probBefore":0.8116055306404093,"visibility":"public","createdTime":1695007885799,"isCancelled":true,"isChallenge":false,"orderAmount":100,"isRedemption":false,"userUsername":"acc","userAvatarUrl":"https://firebasestorage.googleapis.com/v0/b/mantic-markets.appspot.com/o/user-images%2FAccelerationBot%2FtGI0YIHNo5.44?alt=media&token=b4b3edf9-839c-4f1f-941e-82b308eedd5e"},{"id":"XDfHX0feU2544ZavEbod","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":25,"shares":31.39667902325899,"timestamp":1695007885518,"matchedBetId":null}],"isApi":false,"amount":25,"isAnte":false,"shares":31.39667902325899,"userId":"ZWjCvBqdTgSDdpgPC4Tt774AcFe2","outcome":"YES","isFilled":true,"userName":"Bexexexe","probAfter":0.8116055306404093,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":3.5433250352136008,"probBefore":0.7800000000000002,"visibility":"public","createdTime":1695007885518,"isCancelled":false,"isChallenge":false,"orderAmount":25,"isRedemption":false,"userUsername":"bexexexe","userAvatarUrl":"https://lh3.googleusercontent.com/a/ACg8ocL0gc4s5lkz-fLYO9R9m4Kz9oM9_gQLG7AN1IHyPukt0w=s96-c"},{"id":"Bl0n6hTz8tTUM1JuTYhi","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":8.137424988286812,"shares":40.85113229030793,"timestamp":1695005956792,"matchedBetId":null}],"isApi":true,"amount":8.137424988286812,"isAnte":false,"shares":40.85113229030793,"userId":"BhNkw088bMNwIFF2Aq5Gg9NTPzz1","outcome":"NO","isFilled":false,"userName":"Acceleration","limitProb":0.78,"probAfter":0.7800000000000002,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":0,"probBefore":0.8200800115186206,"visibility":"public","createdTime":1695005956792,"isCancelled":true,"isChallenge":false,"orderAmount":100,"isRedemption":false,"userUsername":"acc","userAvatarUrl":"https://firebasestorage.googleapis.com/v0/b/mantic-markets.appspot.com/o/user-images%2FAccelerationBot%2FtGI0YIHNo5.44?alt=media&token=b4b3edf9-839c-4f1f-941e-82b308eedd5e"},{"id":"6mhorKAhQE3q5WeE3rV0","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":100,"shares":134.35220833188305,"timestamp":1695005956590,"matchedBetId":null}],"isApi":false,"amount":100,"isAnte":false,"shares":134.35220833188305,"userId":"uDdVGY4BQ2hv2TNRm163H9KqQ8j1","outcome":"YES","isFilled":true,"userName":"Jame","probAfter":0.8200800115186206,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":0,"probBefore":0.6500000000000001,"visibility":"public","createdTime":1695005956590,"isCancelled":false,"isChallenge":false,"orderAmount":100,"isRedemption":false,"userUsername":"Ja26a","userAvatarUrl":"https://firebasestorage.googleapis.com/v0/b/mantic-markets.appspot.com/o/user-images%2FJa26a%2FDPVnUAUtHd.jpeg?alt=media&token=3c8978ff-d9c8-402a-8927-6a5738f2732a"},{"id":"Zqb5VTzKAIiwErusk2Do","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":2.8945514160341,"shares":8.43967913415193,"timestamp":1694921018138,"matchedBetId":null}],"isApi":true,"amount":2.8945514160341,"isAnte":false,"shares":8.43967913415193,"userId":"BhNkw088bMNwIFF2Aq5Gg9NTPzz1","outcome":"NO","isFilled":false,"userName":"Acceleration","limitProb":0.65,"probAfter":0.6500000000000001,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":0,"probBefore":0.6639918870128059,"visibility":"public","createdTime":1694921018138,"isCancelled":true,"isChallenge":false,"orderAmount":100,"isRedemption":false,"userUsername":"acc","userAvatarUrl":"https://firebasestorage.googleapis.com/v0/b/mantic-markets.appspot.com/o/user-images%2FAccelerationBot%2FtGI0YIHNo5.44?alt=media&token=b4b3edf9-839c-4f1f-941e-82b308eedd5e"},{"id":"NlcCwAijlw6yM7bSNez3","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":20,"shares":31.375616393638893,"timestamp":1694921017954,"matchedBetId":null}],"isApi":false,"amount":20,"isAnte":false,"shares":31.375616393638893,"userId":"uDdVGY4BQ2hv2TNRm163H9KqQ8j1","outcome":"YES","isFilled":true,"userName":"Jame","probAfter":0.6639918870128059,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":0,"probBefore":0.6100000000000002,"visibility":"public","createdTime":1694921017954,"isCancelled":false,"isChallenge":false,"orderAmount":20,"isRedemption":false,"userUsername":"Ja26a","userAvatarUrl":"https://firebasestorage.googleapis.com/v0/b/mantic-markets.appspot.com/o/user-images%2FJa26a%2FDPVnUAUtHd.jpeg?alt=media&token=3c8978ff-d9c8-402a-8927-6a5738f2732a"},{"id":"YP2ojYcWumcMh6kzR4Ll","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":14.377003321685033,"shares":41.34814466364287,"timestamp":1694104886452,"matchedBetId":null}],"isApi":true,"amount":14.377003321685033,"isAnte":false,"shares":41.34814466364287,"userId":"BhNkw088bMNwIFF2Aq5Gg9NTPzz1","outcome":"NO","isFilled":false,"userName":"Acceleration","limitProb":0.61,"probAfter":0.6100000000000001,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":7.213885769273946,"probBefore":0.6923674119259503,"visibility":"public","createdTime":1694104886452,"isCancelled":true,"isChallenge":false,"orderAmount":100,"isRedemption":false,"userUsername":"acc","userAvatarUrl":"https://firebasestorage.googleapis.com/v0/b/mantic-markets.appspot.com/o/user-images%2FAccelerationBot%2FtGI0YIHNo5.44?alt=media&token=b4b3edf9-839c-4f1f-941e-82b308eedd5e"},{"id":"85pqbkWUcHN7vtANZHTj","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":75,"shares":136.87119174429904,"timestamp":1694104886225,"matchedBetId":null}],"isApi":false,"amount":75,"isAnte":false,"shares":136.87119174429904,"userId":"uDdVGY4BQ2hv2TNRm163H9KqQ8j1","outcome":"YES","isFilled":true,"userName":"Jame","probAfter":0.6923674119259503,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":51.407391621366486,"probBefore":0.3956777058057983,"visibility":"public","createdTime":1694104886225,"isCancelled":false,"isChallenge":false,"orderAmount":75,"isRedemption":false,"userUsername":"Ja26a","userAvatarUrl":"https://firebasestorage.googleapis.com/v0/b/mantic-markets.appspot.com/o/user-images%2FJa26a%2FDPVnUAUtHd.jpeg?alt=media&token=3c8978ff-d9c8-402a-8927-6a5738f2732a"},{"id":"QQwZctvAD9R31wU7pV13","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[],"isApi":false,"amount":0,"isAnte":false,"shares":0,"userId":"KyL8Ne9ytVZOcOcfHFxF9DacOgL2","outcome":"YES","isFilled":false,"userName":"Christopher Randles","limitProb":0.33,"probAfter":0.39567770580579836,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":0,"probBefore":0.39567770580579836,"visibility":"public","createdTime":1694060206475,"isCancelled":false,"isChallenge":false,"orderAmount":100,"isRedemption":false,"userUsername":"ChristopherRandles","userAvatarUrl":"https://lh3.googleusercontent.com/a/AGNmyxY3_PAyuLB3J_OqBJQ5T3NjkNLcX7WEzxb7X0Fw=s96-c"},{"id":"RcX9dPSp54EyDnm5wBBB","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":27.599924541909996,"shares":59.26525223092813,"timestamp":1694045334109,"matchedBetId":null},{"amount":61.111111111111114,"shares":111.11111111111111,"timestamp":1694045334111,"matchedBetId":"zA4hD3cBxqN2y9tgIQ2t"},{"amount":11.288964346978887,"shares":19.55199669439041,"timestamp":1694045334111,"matchedBetId":null}],"isApi":false,"amount":100,"isAnte":false,"shares":189.92836003642964,"userId":"Dy1KeRQcZbQgI7oeMHJbsrBpOii2","outcome":"NO","isFilled":true,"userName":"Sorin Tanase Nicola","probAfter":0.39567770580579825,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":32.811235501724,"probBefore":0.6164431409142145,"visibility":"public","createdTime":1694045334111,"isCancelled":false,"isChallenge":false,"orderAmount":100,"isRedemption":false,"userUsername":"SorinTanaseNicola","userAvatarUrl":"https://lh3.googleusercontent.com/a-/AOh14GjUOTnGv06_d41TEAmkKaLChX2qkmdzsAyrh-Rupw=s96-c"},{"id":"zA4hD3cBxqN2y9tgIQ2t","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":50,"shares":111.11111111111111,"timestamp":1694045334111,"matchedBetId":"RcX9dPSp54EyDnm5wBBB"}],"isApi":false,"amount":50,"isAnte":false,"shares":111.11111111111111,"userId":"KyL8Ne9ytVZOcOcfHFxF9DacOgL2","outcome":"YES","isFilled":true,"userName":"Christopher Randles","limitProb":0.45,"probAfter":0.6164431409142146,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":0,"probBefore":0.6164431409142146,"visibility":"public","createdTime":1694036402682,"isCancelled":false,"isChallenge":false,"orderAmount":50,"isRedemption":false,"userUsername":"ChristopherRandles","userAvatarUrl":"https://lh3.googleusercontent.com/a/AGNmyxY3_PAyuLB3J_OqBJQ5T3NjkNLcX7WEzxb7X0Fw=s96-c"},{"id":"O5cpu7DxtdjjCV1dKgWG","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":25,"shares":46.702945713246486,"timestamp":1694036280651,"matchedBetId":null}],"isApi":false,"amount":25,"isAnte":false,"shares":46.702945713246486,"userId":"KyL8Ne9ytVZOcOcfHFxF9DacOgL2","outcome":"YES","isFilled":true,"userName":"Christopher Randles","probAfter":0.6164431409142146,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":31.027217710251037,"probBefore":0.45248868778280527,"visibility":"public","createdTime":1694036280651,"isCancelled":false,"isChallenge":false,"orderAmount":25,"isRedemption":false,"userUsername":"ChristopherRandles","userAvatarUrl":"https://lh3.googleusercontent.com/a/AGNmyxY3_PAyuLB3J_OqBJQ5T3NjkNLcX7WEzxb7X0Fw=s96-c"},{"id":"lqvsRhQ9BMiIGKD21dtg","fees":{"creatorFee":0,"platformFee":0,"liquidityFee":0},"fills":[{"amount":5,"shares":9.545454545454533,"timestamp":1694014711363,"matchedBetId":null}],"isApi":false,"amount":5,"isAnte":false,"shares":9.545454545454533,"userId":"8F51KnOVikcy57eEkJ2QxMEjSqZ2","outcome":"NO","isFilled":true,"userName":"Scott Supak","probAfter":0.4524886877828055,"contractId":"irIFAGnxjQynchTNRiMx","loanAmount":1.6458640078158844,"probBefore":0.5,"visibility":"public","createdTime":1694014711363,"isCancelled":false,"isChallenge":false,"orderAmount":5,"isRedemption":false,"userUsername":"ScottSupak","userAvatarUrl":"https://lh3.googleusercontent.com/a/AEdFTp5DFv99tA_zCypikjGaDt5DPMJL0KdMjFQR3rdlyg=s96-c"}]"""

slug_json = json.loads(jsonstr, strict = False)
bets_json = json.loads(bets_str, strict = False)
pool = slug_json['pool']
p = slug_json['p']

In [342]:
def getCpmmProbability(pool, p):
    YES = pool['YES']
    NO = pool['NO']
    return (p * NO) / (((1 - p) * YES) + (p * NO))

def addCpmmLiquidity(pool, p):
    prob = getCpmmProbability(pool, p)
    y = pool['YES']
    n = pool['NO']
    numerator = y * prob
    denominator = (y * prob) - (n * (prob - 1))
    post_bet_p = numerator / denominator
    return post_bet_p

# simplified call for no limit order code
def calculateCpmmPurchase_Simplified(pool, p, bet):
    bet_amount = bet['amount']
    bet_outcome = bet['outcome']
    bet_shares = calculateCpmmShares(pool, p, bet_outcome, bet_amount)
    bet['shares'] = bet_shares

    y = pool['YES']
    n = pool['NO']
    if bet_outcome == 'YES':
        newY = y - bet_shares + bet_amount
        newN = n + bet_amount
    else:
        newY = y + bet_amount
        newN = n - bet_shares + bet_amount
    post_bet_pool = {}
    post_bet_pool['YES'] = newY
    post_bet_pool['NO'] = newN
    post_bet_p = addCpmmLiquidity(post_bet_pool, p)
    return bet, post_bet_pool, post_bet_p

# TODO: NOT IMPLEMENTED
def getCpmmFees(state, bet, outcome):
    return None

def calculateCpmmPurchase(state, bet_amount, bet_outcome):
    pool = state['pool']
    p = state['p']
    # not implementing fees
    # [remainingBet, fees] = getCpmmFees(state, bet, outcome)
    fees = 0
    
    bet_shares = calculateCpmmShares(pool, p, bet_outcome, bet_amount)

    y = pool['YES']
    n = pool['NO']
    if bet_outcome == 'YES':
        newY = y - bet_shares + bet_amount
        newN = n + bet_amount
    else:
        newY = y + bet_amount
        newN = n - bet_shares + bet_amount
    post_bet_pool = {}
    post_bet_pool['YES'] = newY
    post_bet_pool['NO'] = newN
    post_bet_p = addCpmmLiquidity(post_bet_pool, p)
    return bet_shares, post_bet_pool, post_bet_p, fees

# before liquidity fee
def calculateCpmmShares(pool, p, bet_outcome, bet_amount):
    y = pool['YES']
    n = pool['NO']
    k = (y ** p) * (n ** (1 - p))
    if bet_outcome == 'YES':
        shares = y + bet_amount - (k * ((bet_amount + n) ** (p - 1))) ** (1 / p)
    else:
        shares = n + bet_amount - (k * ((bet_amount + y) ** -p)) ** (1 / (1 - p))
    return shares

def getProbability(pool, p):
    return (p * pool['NO']) / (((1 - p) * pool['YES']) + (p * pool['NO']));

def calculateFixedMktPayout(p, bet):
    bet_outcome = bet['outcome']
    shares = bet['shares']
    if bet_outcome == 'YES':
        betP = p
    else:
        betP = 1 - p
    return betP * shares

bet = {}
bet['outcome'] = 'YES'

bet['amount'] = 200
[bet, post_bet_pool, post_bet_p] = calculateCpmmPurchase_Simplified(pool, p, bet)
calculateFixedMktPayout(post_bet_p, bet)
prob = getCpmmProbability(post_bet_pool, post_bet_p)
print(f"Original pool: {pool}")
print(f"Original p: {p}")
print(f"Post bet pool: {post_bet_pool}")
print(f"Post bet p: {post_bet_p}")
print(f"Post bet probability: {prob}")
print(f"Bet: {bet}")

Original pool: {'NO': 289.4503968614656, 'YES': 159.27045244798285}
Original p: 0.6125392284511574
Post bet pool: {'YES': 114.24285686757833, 'NO': 489.4503968614656}
Post bet p: 0.6125392284511574
Post bet probability: 0.871350762289236
Bet: {'outcome': 'YES', 'amount': 200, 'shares': 245.02759558040452}


In [343]:
import copy
def get_unfilled_bets_query(contract_doc):
    unfilled_contract_doc = copy.deepcopy(contract_doc)
    unfilled_contract_doc['bets'] = []
    for bet in contract_doc['bets']:
        if 'isFilled' in bet and 'isCancelled' in bet:
            if bet['isFilled'] is False and bet['isCancelled'] is False:
                unfilled_contract_doc['bets'].append(bet)
    return unfilled_contract_doc

def get_unfilled_bets_and_user_balances(contract_doc, bettor_id):
    unfilled_bets = get_unfilled_bets_query(contract_doc)['bets']
    # Get balance of all users with open limit orders.
    user_ids = list(set([bet['userId'] for bet in unfilled_bets]))
    # similar to all_users but only a select subset
    select_users = get_user_ids_data(user_ids, all_users_filepath)
    user = {}
    balance_by_user_id = {user_id: user_data['balance'] for user_id, user_data in select_users.items()}
    return {'unfilledBets': unfilled_bets, 'balanceByUserId': balance_by_user_id}

def update_makers(makers, taker_bet_id, contract_doc, trans):
    from collections import defaultdict
    from operator import itemgetter
    from functools import reduce
    from firestore_batch import Batch

    makers_by_bet = defaultdict(list)
    for maker in makers:
        makers_by_bet[maker['bet']['id']].append(maker)

    batch = Batch(trans)

    for makers in makers_by_bet.values():
        bet = makers[0]['bet']
        new_fills = [
            {
                'amount': maker['amount'],
                'shares': maker['shares'],
                'matchedBetId': taker_bet_id,
                'timestamp': maker['timestamp']
            }
            for maker in makers
        ]

        fills = bet.get('fills', []) + new_fills
        total_shares = reduce(lambda acc, fill: acc + fill['shares'], fills, 0)
        total_amount = reduce(lambda acc, fill: acc + fill['amount'], fills, 0)
        is_filled = total_amount == bet['orderAmount']

        log('Updated a matched limit order.')
        contract_doc['bets']['id'].update({
            'fills': fills,
            'isFilled': is_filled,
            'amount': total_amount,
            'shares': total_shares,
        })

    # Deduct balance of makers.
    spent_by_user = defaultdict(float)
    for maker in makers:
        spent_by_user[maker['bet']['userId']] += maker['amount']

    for user_id, spent in spent_by_user.items():
        batch.update(firestore.document(f'users/{user_id}'), {'balance': firestore.Increment(-spent)})

    batch.commit()

# Usage example:
# await update_makers(makers_list, taker_bet_id, contract_doc, transaction)


In [346]:
#getBinaryCpmmBetInfo(slug_json, )
#def getBinaryCpmmBetInfo(contract: Dict, outcome: str, betAmount: float, limitProb: float, 
#    unfilledBets: List[LimitBet], balanceByUserId: Dict[str, float], expiresAt=None) -> Dict:

def calculate_new_bet_info(outcome_type, mechanism, uid, contract_doc, contract, bet):
    if (
        (outcome_type == 'BINARY' or outcome_type == 'PSEUDO_NUMERIC' or outcome_type == 'STONK') and
        mechanism == 'cpmm-1'
    ):
        #outcome, limit_prob, expires_at = validate(binary_schema, body)
        # limit_prob is for limit orders (NOT IMPLEMENTED)
        limit_prob = None
        # expires_at is for limit orders (NOT IMPLEMENTED)
        expires_at = None
        
        if limit_prob is not None and outcome_type == 'BINARY':
            is_rounded = floating_equal(round(limit_prob * 100), limit_prob * 100)
            if not is_rounded:
                raise APIError(
                    400,
                    'limitProb must be in increments of 0.01 (i.e. whole percentage points)'
                )
            limit_prob = round(limit_prob * 100) / 100

        """
        print(
            f"Checking for limit orders in placebet for user {uid} on contract id {contract_doc['id']}."
        )
        """
        result = get_unfilled_bets_and_user_balances(
            contract_doc, uid
        )
        unfilled_bets = result['unfilledBets']
        balance_by_user_id = result['balanceByUserId']
        
        return getBinaryCpmmBetInfo(
            contract,
            bet['outcome'],
            bet['amount'],
            limit_prob,
            unfilled_bets,
            balance_by_user_id,
            expires_at
        )
    elif (
        (outcome_type == 'FREE_RESPONSE' or outcome_type == 'MULTIPLE_CHOICE') and
        mechanism == 'dpm-2'
    ):
        answer_id = bet['answerId']
        answer_doc = contract_doc['answers'][answer_id]
        
        return get_new_multi_bet_info(answer_id, amount, contract)
    elif outcome_type == 'MULTIPLE_CHOICE' and mechanism == 'cpmm-multi-1':
        #data = validate(multiple_choice_schema, body)
        answer_id = bet['answerId']
        outcome = bet['outcome']
        # for limit orders not used
        #limit_prob = data.get('limitProb')
        limit_prob = None
        # for limit orders not used
        #expires_at = data.get('expiresAt')
        expires_at = None

        answers = contract_doc['answers']
        answer = next((a for a in answers if a['id'] == answer_id), None)
        if not answer:
            raise APIError(400, 'Invalid answerId')

        result = get_unfilled_bets_and_user_balances(
            contract_doc, uid
        )
        unfilled_bets = result['unfilledBets']
        balance_by_user_id = result['balanceByUserId']

        return get_new_multi_cpmm_bet_info(
            contract,
            answers,
            answer,
            outcome,
            amount,
            limit_prob,
            unfilled_bets,
            balance_by_user_id,
            expires_at
        )
    elif outcome_type == 'NUMERIC' and mechanism == 'dpm-2':
        #data = validate(numeric_schema, body)
        outcome = bet['outcome']
        value = bet['value']
        amount = bet['amount']
        return get_numeric_bets_info(value, outcome, amount, contract)
    else:
        raise APIError(500, 'Contract has an invalid type/mechanism.')

outcome_type = slug_json['outcomeType']
mechanism = slug_json['mechanism']
uid = slug_json['id']
contract = slug_json
contract_doc = {
    'bets': bets_json,
    'id': uid
}


bet = {}
bet['outcome'] = 'NO'
bet['amount'] = 385

# Call the async function with an immediately invoked async function expression (IIFE)
result = calculate_new_bet_info(
            outcome_type, mechanism, uid, contract_doc, contract, bet
        )
print(result['newBet']['probAfter'])

0.2867486943406567


In [366]:
def bet_to_shift_to_desired_probability(desiredProb):
    bet = {}
    
    bet['outcome'] = 'NO'
    bet['amount'] = 0
    result = calculate_new_bet_info(
                outcome_type, mechanism, uid, contract_doc, contract, bet
            )
    probNow = result['newBet']['probAfter']
    
    if probNow > desiredProb:
        bet['outcome'] = 'NO'
        increaseProb = False
    else:
        bet['outcome'] = 'YES'
        increaseProb = True
    
    # max betting amount to search for
    max_amount = 100000
    
    # start search half-way between 0 and max_amount
    lower = 0
    upper = max_amount
    
    newAmount = int(round((lower + upper) / 2.0))
    bet['amount'] = newAmount
    
    lastDeltaProb = 100
    lastDeltaMana = max_amount
    lastProb = probNow
    
    # binary search until deltaMana is 1
    while(not(newAmount == lower or newAmount == upper)):
        result = calculate_new_bet_info(
                    outcome_type, mechanism, uid, contract_doc, contract, bet
                )
        probAfter = result['newBet']['probAfter']
        print(bet)
        print("  ", probAfter)
        deltaProb = probAfter - desiredProb
        if deltaProb >= 0:
            positiveDeltaProb = True
        else:
            positiveDeltaProb = False
            
        if positiveDeltaProb:
            if increaseProb:
                # too far over for YES
                upper = bet['amount']
            else:
                # haven't bet enough for NO
                lower = bet['amount']
        else:
            if increaseProb:
                # haven't bet enough for YES
                lower = bet['amount']
            else:
                # too far under for NO
                upper = bet['amount']
        
        newAmount = int(round((lower + upper) / 2.0))
        bet['amount'] = newAmount

In [368]:
print(bet_to_shift_to_desired_probability(0.40))

{'outcome': 'NO', 'amount': 50000}
   1.035688327245512e-06
{'outcome': 'NO', 'amount': 25000}
   6.2106829968719775e-06
{'outcome': 'NO', 'amount': 12500}
   3.7327013030107575e-05
{'outcome': 'NO', 'amount': 6250}
   0.0002253286900920028
{'outcome': 'NO', 'amount': 3125}
   0.001371449448290765
{'outcome': 'NO', 'amount': 1562}
   0.008460923772406405
{'outcome': 'NO', 'amount': 781}
   0.05218471180351717
{'outcome': 'NO', 'amount': 390}
   0.27913228980189364
{'outcome': 'NO', 'amount': 195}
   0.33
{'outcome': 'NO', 'amount': 98}
   0.4545685543946975
{'outcome': 'NO', 'amount': 146}
   0.34893026570067426
{'outcome': 'NO', 'amount': 122}
   0.3983328740003131
{'outcome': 'NO', 'amount': 110}
   0.4255849169372906
{'outcome': 'NO', 'amount': 116}
   0.41174282335656515
{'outcome': 'NO', 'amount': 119}
   0.4049839844610984
{'outcome': 'NO', 'amount': 120}
   0.40275499753468536
{'outcome': 'NO', 'amount': 121}
   0.4005379661813049
None


In [369]:
print(bet_to_shift_to_desired_probability(0.20))

{'outcome': 'NO', 'amount': 50000}
   1.035688327245512e-06
{'outcome': 'NO', 'amount': 25000}
   6.2106829968719775e-06
{'outcome': 'NO', 'amount': 12500}
   3.7327013030107575e-05
{'outcome': 'NO', 'amount': 6250}
   0.0002253286900920028
{'outcome': 'NO', 'amount': 3125}
   0.001371449448290765
{'outcome': 'NO', 'amount': 1562}
   0.008460923772406405
{'outcome': 'NO', 'amount': 781}
   0.05218471180351717
{'outcome': 'NO', 'amount': 390}
   0.27913228980189364
{'outcome': 'NO', 'amount': 586}
   0.10846632352250861
{'outcome': 'NO', 'amount': 488}
   0.16910044104578076
{'outcome': 'NO', 'amount': 439}
   0.21578832456698852
{'outcome': 'NO', 'amount': 464}
   0.1902055387570674
{'outcome': 'NO', 'amount': 452}
   0.20198975597235833
{'outcome': 'NO', 'amount': 458}
   0.19598802808298219
{'outcome': 'NO', 'amount': 455}
   0.19896093879664803
{'outcome': 'NO', 'amount': 454}
   0.1999642777883379
{'outcome': 'NO', 'amount': 453}
   0.20097386969535017
None
